# AutoQuant

This notebook shows a working code example of how to use AIMET AutoQuant feature.

AIMET offers a suite of neural network post-training quantization (PTQ) techniques that can be applied in succession. However, the process of finding the right combination and sequence of techniques to apply is time-consuming and requires careful analysis, which can be challenging especially for non-expert users. We instead recommend AutoQuant to save time and effort.

AutoQuant is an API that applies various PTQ techniques in AIMET automatically based on analyzing the model and best-known heuristics. In AutoQuant, users specify the amount of tolerable accuracy drop, and AutoQuant will apply PTQ techniques cumulatively until the target accuracy is satisfied.


#### Overall flow
This notebook covers the following
1. Define constants and helper functions
2. Load a pretrained FP32 model
3. Run AutoQuant

#### What this notebook is not
This notebook is not designed to show state-of-the-art AutoQuant results. For example, it uses a relatively quantization-friendly model like Resnet18. Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly.


---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [2]:
import cv2
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

transform = transforms.Compose([
            transforms.ToTensor(),  # Convert to tensor
            transforms.Resize([224, 224]),  # Resize
        ])

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, 0

calibrationDataset = CustomImageDataset(root_dir='/datasets/COCO/images', transform=transform)
# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

## 1. Define Constants and Helper functions

In this section the constants and helper functions needed to run this eaxmple are defined.

- **EVAL_DATASET_SIZE** A typical value is 5000. In this notebook, this value has been set to 500 for faster execution.
- **CALIBRATION_DATASET_SIZE** A typical value is 2000. In this notebook, this value has been set to 200 for faster execution.


The helper function **_create_sampled_data_loader()** returns a DataLoader based on the dataset and the number of samples provided.

In [3]:
import random
from typing import Optional
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, Subset
from aimet_torch.utils import in_eval_mode, get_device
from mmdet.apis import inference_detector
import numpy as np

EVAL_DATASET_SIZE = 500
CALIBRATION_DATASET_SIZE = 200
BATCH_SIZE = 1

_datasets = {}

def _create_sampled_data_loader(dataset, num_samples):
    if num_samples not in _datasets:
        indices = random.sample(range(len(dataset)), num_samples)
        _datasets[num_samples] = Subset(dataset, indices)
    return DataLoader(_datasets[num_samples], batch_size=BATCH_SIZE)


def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = EVAL_DATASET_SIZE

    data_loader = _create_sampled_data_loader(calibrationDataset, 10)
    device = get_device(model)

    correct = 0
    for image, label in tqdm(data_loader):
        image = image.cpu().detach().numpy()[0]
        image = image.transpose(1, 2, 0)
        losses = inference_detector(model, image)
        # for l in losses['loss_bbox']:
        #     bbox_loss += l

    return correct / num_samples

2024-07-25 12:37:09,324 - root - INFO - AIMET


## 2. Load a pretrained FP32 model
For this example, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [5]:
# %cd /content/drive/MyDrive/Aimet-torch/mmdetection-3.3.0/

import torch

from mmdet.apis import init_detector

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = 'rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

model = init_detector(CONFIG_PATH, WEIGHTS_PATH, device=DEVICE)


Loads checkpoint by local backend from path: rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



In [6]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
from aimet_torch.model_preparer import prepare_model

# quant_model = QuantizationSimModel(model, torch.rand((1, 3, 224, 224)))

dummy_input = torch.rand(1, 3, 640, 640)  # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
# # if use_cuda:
# #     dummy_input = dummy_input.cuda()

print("constructor called for Quant Sim Model")

sim_model = prepare_model(model)

# sim = QuantizationSimModel(model=sim_model,
#                         quant_scheme=QuantScheme.post_training_tf_enhanced,
#                         dummy_input=dummy_input,
#                         default_output_bw=8,
#                         default_param_bw=8,)

constructor called for Quant Sim Model
2024-07-25 12:38:22,579 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.0.bn.module_batch_norm} 
2024-07-25 12:38:22,580 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.1.bn.module_batch_norm_1} 
2024-07-25 12:38:22,580 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.2.bn.module_batch_norm_2} 
2024-07-25 12:38:22,581 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.0.bn.module_batch_norm_3} 
2024-07-25 12:38:22,581 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.short_conv.bn.module_batch_norm_4} 
2024-07-25 12:38:22,581 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.main_conv.bn.module_batch_norm_5} 
2024-07-25 12:38:22,582 - ModelPreparer - INFO - Functional         : Adding new module for node

In [7]:
accuracy = eval_callback(model)
print(f'- FP32 accuracy: {accuracy}')

100%|██████████| 10/10 [00:00<00:00, 14.33it/s]

- FP32 accuracy: 0.0


In [ ]:
# import torch
# from mmdet.apis import DetInferencer

# inferencer = DetInferencer(model='rtmdet_tiny_8xb32-300e_coco')
# model = inferencer.model
# if torch.cuda.is_available():
#     model.to(torch.device('cuda'))

# accuracy = eval_callback(model)
# print(f'- FP32 accuracy: {accuracy}')

In [ ]:
# from torchvision.models import resnet18

# model = resnet18(pretrained=True).eval()


## 3. Run AutoQuant
### Create AutoQuant Object

The AutoQuant feature utilizes an unlabeled dataset to achieve quantization. The class **UnlabeledDatasetWrapper** creates an unlabeled Dataset object from a labeled Dataset.

In [10]:
from aimet_torch.auto_quant import AutoQuant

class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset):
        self._dataset = dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index):
        images, _ = self._dataset[index]
        return images


data_loader = _create_sampled_data_loader(UnlabeledDatasetWrapper(calibrationDataset), 10)
def new_eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = EVAL_DATASET_SIZE

    correct = 0
    for image in tqdm(data_loader):
        image = image[0].cpu().detach()
        model(image)
        # for l in losses['loss_bbox']:
        #     bbox_loss += l

    return correct / num_samples


auto_quant = AutoQuant(sim_model,
                       dummy_input=dummy_input,
                       data_loader=data_loader,
                       eval_callback=new_eval_callback)

### Run AutoQuant Inference
This step runs AutoQuant inference. AutoQuant inference will run evaluation using the **eval_callback** with the vanilla quantized model without applying PTQ techniques. This will be useful for measuring the baseline evaluation score before running AutoQuant optimization.

In [11]:
sim, initial_accuracy = auto_quant.run_inference()
print(f"- Quantized A ccuracy (before optimization): {initial_accuracy}")

2024-07-25 12:39:44,414 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm} 


2024-07-25 12:39:44,416 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul} 


2024-07-25 12:39:44,417 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_1} 


2024-07-25 12:39:44,418 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul_1} 


2024-07-25 12:39:44,419 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_2} 


...

2024-07-25 12:40:03,776 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.000000)


| Batchnorm Folding


2024-07-25 12:40:06,174 - Quant - INFO - No config file provided, defaulting to config file at /home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2024-07-25 12:40:06,189 - Quant - INFO - Unsupported op type Squeeze
2024-07-25 12:40:06,189 - Quant - INFO - Unsupported op type Mean
2024-07-25 12:40:06,196 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


100%|██████████| 10/10 [00:02<00:00,  4.56it/s]


- Quantized A ccuracy (before optimization): 0.0


### Set AdaRound Parameters (optional)
AutoQuant uses a set of predefined default parameters for AdaRound.
These values were determined empirically and work well with the common models.
However, if necessary, you can also use your custom parameters for Adaround.
In this notebook, we will use very small AdaRound parameters for faster execution.

In [12]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

ADAROUND_DATASET_SIZE = 10
adaround_data_loader = _create_sampled_data_loader(calibrationDataset, ADAROUND_DATASET_SIZE)
adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader), default_num_iterations=2000)
auto_quant.set_adaround_params(adaround_params)

### Run AutoQuant Optimization
This step runs AutoQuant optimization, which returns the best possible quantized model, corresponding evaluation score and the path to the encoding file.
The **allowed_accuracy_drop** parameter indicates the tolerable amount of accuracy drop. AutoQuant applies a series of quantization features until the target accuracy (FP32 accuracy - allowed accuracy drop) is satisfied. When the target accuracy is reached, AutoQuant will return immediately without applying furhter PTQ techniques. Please refer AutoQuant User Guide and API documentation for complete details.

In [13]:
model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy}")

2024-07-25 12:40:39,894 - AutoQuant - INFO - Starting AutoQuant


100%|██████████| 10/10 [00:00<00:00, 55.17it/s]

2024-07-25 12:40:40,079 - AutoQuant - INFO - Target eval score: -0.010000
2024-07-25 12:40:40,079 - AutoQuant - INFO - FP32 eval score (W32A32): 0.000000



- Prepare Model


2024-07-25 12:40:43,534 - Quant - INFO - No config file provided, defaulting to config file at /home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json


...

2024-07-25 12:41:30,628 - AutoQuant - INFO - Evaluation finished: W32A8 (eval score: 0.000000)


/ W32 Evaluation


2024-07-25 12:41:30,892 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.000000)


- Batchnorm Folding

2024-07-25 12:41:31,385 - AutoQuant - INFO - Best eval score: 0.000000
- Quantized Accuracy (after optimization):  0.0


In [ ]:
auto_quant.dummy_input.shape

from aimet_torch.quantsim import QuantizationSimModel

model = QuantizationSimModel(model, auto_quant.dummy_input)

---
## Summary

Hope this notebook was useful for you to understand how to use AIMET AutoQuant feature.

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and parameters
- Refer to the other example notebooks to understand how to use AIMET CLE and AdaRound features in a standalone fashion.